# Example 0

---
#### This notebook will go over a few of the basics of generating new physics events with DarkNews.

Table of Contents:

1) **Generation**
    * generating events with the GenLauncher class
    * generating events on the command line

2) **Output**
    * Main Pandas Dataframe
    * Other formats printed to file
        * loading pickled and parquet pandas dataframes
        * loading numpy arrays with event information
        * hepevt and hepmc3 printing (**Under development!**)

3) **Inputs**
    * Experiments
    * Model parameters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

import DarkNews as dn

## 1) Generation

Let us start by generating neutrino upscattering events using GenLauncher and then using the command line.

### `GenLauncher` class

It is possible to run the generator through an instance of the `GenLauncher` class in the `DarkNews.GenLauncher` module.
We create an instance of this class, specifying the parameters of the run, then we use the `run()` method on this instance.
The created dataset can be found in the `data/` directory tree.
Alternatively, it an be accessed via the `df` attribute.

When using `GenLauncher`, the `run()` method accepts parameters as `log`, `verbose` or `logfile` (which can be also specified during the construction of the object).

As an example, we start by considering a 3+1 model with a heavy Z'. In this case, the HNL decays are 3-body ones. In short, we compute

$$ \nu_\mu A \to (N_4 \to \nu e^+e^-) A $$ 

where $\nu$ are all final state light neutrinos.


In [ ]:
from DarkNews import GenLauncher

gen_object = GenLauncher(mzprime=1.25, m4=0.140, neval=1000, nu_flavors=['nu_mu'], noHF=True, HNLtype="dirac", loglevel="INFO")
df_1 = gen_object.run()

---
We may consider another example, a 3+2 model with a light Z', and compute

$$ \nu_\mu A \to N_5 A \to N_4 Z^\prime A \to N_4 e^+e^- A $$ 

neglecting upscattering to $N_4$. In this case, the HNL decays are 2-body decays. By simply passing a value for the N5 mass, DarkNews already knows to use a 3+2 model, however, by default, all N6 mixing elements are set to zero, so it is important to specify these when using 3 HNLs.

In [ ]:
gen_object_2 = GenLauncher(Umu5=1e-3, UD5=1/np.sqrt(2), chi=0.0031, gD=2, mzprime=0.03, m4=0.080, m5=0.140, neval=1000, HNLtype="dirac")
df_2 = gen_object_2.run(loglevel="INFO")

---

Finally, let's try setting some model-independent interaction vertices.

In [ ]:
# kwargs = {'d_mu5': 1e-4, 'd_45': 0.1, 'deV': 1e-3, 'deA': 1e-3, 'duV': 2e-3, 'ddV': 1e-3}
kwargs = {'d_mu5': 1e-4, 'd_45': 0.1, 'deV': 1e-3, 'deA': 1e-3, 'dprotonV': 1e-3, 'loglevel': "ERROR"}
gen_object_3 = dn.GenLauncher(mzprime=0.03, m4=0.080, m5=0.140, neval=1000, HNLtype="dirac", **kwargs)
df_3 = gen_object_3.run()

---
### `dn_gen` command line functionality

It is also possible to run the generator in the command line via the `dn_gen` script passing the parameters as arguments.
The created dataset can be found in the `data/` directory tree, which is created in the same folder the script is run.

Let's try to run another example. We are running this from the jupyter notebook, but you may as well run it from your own command line.


In [ ]:
cmd_string = "dn_gen --mzprime=0.03 --m4=0.140 --alpha_epsilon2=2e-10 --Umu4=1e-4 --UD4=1 --alphaD=0.25 --neval=1000 --HNLtype=dirac --loglevel=WARNING --hepevt --numpy --pandas --parquet"
_=os.system(cmd_string)

---
# 2) Output format

### Pandas DataFrame Output

Let's start by looking at the main DarkNews dataframe returned by `GenLancher`

Our main object is a `MultiIndex dataframe`, with:
* rows corresponding to individual events 

    * Each event contains the components for all the 4-momenta of the particles involved:
$$\nu _\text{P\_projectile} + \text{Hadronic target} _\text{P\_target} \to N _\text{P\_decay\_N\_parent} + \text{Hadronic recoil} _\text{P\_recoil}$$


* Columns are properties, including four momenta, event rate weights, event scattering regime, scattering and decay positions, scattering target, etc.
* Subcolumns correspond to Lorentz indices for four momenta and position 4-vectors.
* metadata information inside `df.attrs`


| **Column**            | **Subcolumn** |**type**  | **description**|
|:--------------------------|:--------:|:--------:|:-----------------------------------|
| **P\_projectile**         | 0, 1, 2, 3  | `float`  | 4-momenta of beam neutrino |
| **P\_decay\_N\_parent**   | 0, 1, 2, 3  | `float`  | 4-momenta of HNL\_parent |
| **P\_target**             | 0, 1, 2, 3  | `float`  | 4-momenta of nucleus |
| **P\_recoil**             | 0, 1, 2, 3  | `float`  | 4-momenta of recoiled nucleus |
| **P\_decay\_photon**      | 0, 1, 2, 3  | `float`  | 4-momenta of photon (if it exists)|
| **P\_decay\_ell\_minus**  | 0, 1, 2, 3  | `float`  | 4-momenta of e- (if it exists)|
| **P\_decay\_ell\_plus**   | 0, 1, 2, 3  | `float`  | 4-momenta of e+ (if it exists)|
| **P\_decay\_N\_daughter** | 0, 1, 2, 3  | `float`  | 4-momenta of HNL\_daughter / nu\_daughter |
| **pos_scatt**             | 0, 1, 2, 3  | `float`  | upscattering position|
| **pos_decay**             | 0, 1, 2, 3  | `float`  | decay position of primary particle (N\_parent) -- no secondary decay position is saved. |
| **w\_decay\_rate\_0**     | <!-- --> | `float`  | Weight of the decay rate of primary unstable particle: &Sigma;<sub>i</sub> w<sub>i</sub> = &Gamma;<sub>N</sub> |
| **w\_decay\_rate\_1**     | <!-- --> | `float`  | Weight of the decay rate of secondary unstable particle: &Sigma;<sub>i</sub> w<sub>i</sub> = &Gamma;<sub>X</sub> |
| **w\_event\_rate**        | <!-- --> | `float`  | Weight for the event rate: &Sigma;<sub>i</sub> w<sub>i</sub> = event rate |
| **w\_flux\_avg\_xsec**    | <!-- --> | `float`  | Weight of the flux averaged cross section: &Sigma;<sub>i</sub> w<sub>i</sub> = int(sigma &sdot; flux) &sdot; exposure |
| **target**                | <!-- --> | *string* | Name of the target object, it will typically be a nucleus |
| **target\_pdgid**         | <!-- --> | `int`    | PDG id of the target |
| **projectile\_pdgid**         | <!-- --> | `int`    | PDG id of the neutrino projectile |
| **scattering\_regime**    | <!-- --> | *string* | Regime can be coherent or p-elastic |
| **helicity**              | <!-- --> | *string* | Helicity process: can be flipping or conserving; flipping is suppressed |
| **underlying\_process**   | <!-- --> | *string* | String of the underlying process, e.g, "nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_in_C12" |


In [ ]:
print(*df_1)

---
### Metadata

In addition to the information about the events, the pickled or GenLauncher-returned pandas dataframe contains metadata on the model, experiment, and data path properties, all stored in [df.attrs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.attrs.html). We have:

1. **experiment**: this is a class DarkNews.experiment.Detector, which contains all the information about the experiment for which the events are generated. It also contains a list of instances of the NuclearTarget class, which contains information about the different scattering targets used.


In [ ]:
keys = list(df_1.attrs['experiment'].__dict__.keys())
print(f"Attributes of Detector class: \n", *keys)
print(f"\nAttributes of NuclearTarget class: \n", *list(df_1.attrs['experiment'].NUCLEAR_TARGETS[0].__dict__))

Now we look at how to obtain the nuclear targets used by the DarkNews generator

In [ ]:
fid_mass = df_1.attrs['experiment'].FIDUCIAL_MASS_PER_TARGET
for target, fid_mass in zip(df_1.attrs['experiment'].NUCLEAR_TARGETS, df_1.attrs['experiment'].FIDUCIAL_MASS_PER_TARGET):
    print(f"{fid_mass:.2f} tonnes of {target.name}")

---
2. **model**: this attribute contains DarkNews.model.Model, which stores all the new physics parameters used in the generation. This includes both low-level information, such as the couplings, say $g_D$, as well as high-level ones, such as the couplings of the $Z^\prime$ to electrons, $d_e^V$ and $d_e^A$.

In [ ]:
print(df_3.attrs.keys())

print(df_3.attrs['N5_ctau0'])

In [ ]:
print("Some low-level parameters:\n", *list(df_1.attrs['model'].__dict__.keys())[:19],' ...')
print("\nSome high-level parameters:\n", *list(df_1.attrs['model'].__dict__.keys())[-14:],' ...')

---
### Other output formats

Let's now look at some other formats printed to file.

In [ ]:
path_to_data = Path('./data/miniboone_fhc/3plus1/m4_0.14_mzprime_0.03_dirac/')

# pickled pandas dataframe 
df_p = pd.read_pickle(path_to_data/'pandas_df.pckl')

# pandas dataframe from parquet
df_p_pq = pd.read_parquet(path_to_data/'pandas_df.parquet')

# numpy ndarray
nd_a = np.load(path_to_data/'ndarray.npy')

# HEPEVT 
import pyhepmc as hep
hepevt_file = hep.ReaderHEPEVT(str(path_to_data/'HEPevt.dat'))
evt = hepevt_file.read_event


In [ ]:
# and now histogram neutrino energy as an example
fig, ax = dn.plot_tools.std_fig(figsize=(10,4))

_=ax.hist(df_p['P_projectile','0'], weights=df_p['w_event_rate',''], bins=50, histtype='step', label='pickled')
_=ax.hist(df_p_pq['P_projectile','0'], weights=df_p_pq['w_event_rate',''], bins=50, ls='--', histtype='step', label='parquet')

# unfortunately, the columns of the numpy array format are not specified and the correspondence is done by hand
_=ax.hist(nd_a[:,0], weights=nd_a[:,-4], bins=50, ls=':', color='violet', histtype='step', label='numpy')

ax.legend()
ax.set_ylabel('Events')
ax.set_xlabel(r'$E_\nu/$GeV')

In [ ]:
# Let's compute something more interesting: opening angle of e+e-

mee = np.arccos(dn.fourvec.df_cos_opening_angle(df_p['P_decay_ell_minus'], df_p['P_decay_ell_plus']))*dn.const.rad_to_deg

fig, ax = dn.plot_tools.std_fig(figsize=(10,4))

_=ax.hist(mee, weights=df_p['w_event_rate',''], bins=50, histtype='step', label='pickled')

ax.legend()
ax.set_ylabel('Events')
ax.set_xlabel(r'$\Delta \theta_{ee}$ (degrees)')

---
# 3) Inputs


### Experiments
It is possible to select an experiment through the `experiment` argument of the `GenLauncher` class (or `--experiment` option of the command line interface):
1. specifying a keyword for a pre-defined experiment among:
    * DUNE FHC ND (`"dune_nd_fhc"`)
    * DUNE RHC ND (`"dune_nd_rhc"`)
    * MicroBooNE (`"microboone"`)
    * MINERVA FHC LE (`"minerva_le_fhc"`)
    * MINERVA FHC ME (`"minerva_me_fhc"`)
    * MINERVA RHC ME (`"minerva_me_rhc"`)
    * MiniBooNE FHC (`"miniboone_fhc"`)
    * MINOS FHC LE (`"minos_le_fhc"`)
    * MINOS FHC ME (`"minos_me_fhc"`)
    * ND280 FHC (`"nd280_fhc"`)
    * NOvA FHC (`"nova_le_fhc"`)
    * NuTeV FHC (`"nutev_fhc"`)
    * FASERv (`"fasernu"`)

2. specifying the file path of an experiment file: every file should be specified using the same rules as for the parameters file.
A template file [`template_custom_experiment.txt`](examples/template_custom_experiment.txt) can be found in this directory.

In [ ]:
list_of_experiments = [
'dune_nd_fhc',
'dune_nd_rhc',
'microboone',
'minerva_le_fhc',
'minerva_me_fhc',
'minerva_me_rhc',
'miniboone_fhc',
'minos_le_fhc',
'nd280_fhc',
'nova_le_fhc',
'nutev_fhc',
'fasernu']

rates = []
for exp in list_of_experiments:
    # use default model 
    print(exp)
    gen_object = GenLauncher(experiment=exp, neval=200, loglevel='ERROR')
    df=gen_object.run()
    rates.append(sum(df.w_event_rate))

Now we print the total rate in each expriment -- note we chose a very small number of evaluations. 

Note that these numbers don't mean anything without an appropriate definition of signal selection and fiducial volume. 

Exposures and target masses do not necessarily represent the official ones used by collaborations, nor are they optimized for this signature.

In [ ]:
for exp,rate in zip(list_of_experiments,rates):
    print(f'{exp}: {rate:.2g}')

---
### Custom Experiment files

And now we can input our own user-defined experimental files.

In [ ]:
cmd_string = "dn_gen --experiment=./template_custom_experiment.txt --loglevel=INFO"
_=os.system(cmd_string)

In [ ]:
gen_object = GenLauncher(experiment="./template_custom_experiment.txt", loglevel='ERROR')
_=gen_object.run()

---
### Model files

It is possible to specify model parameters through a file.
This can be done either via the command-line interface, thorugh the option `--param-file` or via the `GenLauncher` class, thorugh the argument `param-file` in the constructor.
In any case, the parameters specified in the file will overwrite the default ones, but will be overwritten by any other definition (either via options in command-line mode or via the keyword arguments in the `GenLauncher` constructor) of the same variable.

A template file [`template_parameters_file.txt`](examples/template_parameters_file.txt) can be found in this directory.
The file interface allows the user to specify the parameters with far more freedom, enabling the possibility to use mathematical expressions, involving user-defined constants, while keeping all the parameters in one single file.

In [ ]:
cmd_string = "dn_gen --param-file=./parameters_example_1.txt --loglevel=ERROR"
_=os.system(cmd_string)

In [ ]:
gen_object = GenLauncher(param_file="./parameters_example_1.txt", loglevel='ERROR')
df_1 = gen_object.run()

In [ ]:
cmd_string = "dn_gen --param-file=./parameters_example_2.txt --loglevel=ERROR"
_=os.system(cmd_string)

In [ ]:
gen_object = GenLauncher(param_file="./parameters_example_2.txt", loglevel='ERROR')
df_2=gen_object.run()